In [1]:
import os 
import pandas as pd 
from bs4 import BeautifulSoup


In [2]:
!pip3 install html5lib

In [3]:
SCORES_DIR= 'data2/scores'

In [4]:
box_scores= os.listdir(SCORES_DIR)

In [5]:
box_scores = os.listdir(SCORES_DIR)
box_scores = [os.path.join(SCORES_DIR, f) for f in box_scores if f.endswith(".html")]

In [6]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [7]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [8]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [9]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [10]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 2734
200 / 2734
300 / 2734
400 / 2734
500 / 2734
600 / 2734
700 / 2734
800 / 2734
900 / 2734
1000 / 2734
1100 / 2734
1200 / 2734
1300 / 2734
1400 / 2734
1500 / 2734
1600 / 2734
1700 / 2734
1800 / 2734
1900 / 2734
2000 / 2734
2100 / 2734
2200 / 2734
2300 / 2734
2400 / 2734
2500 / 2734
2600 / 2734
2700 / 2734


In [11]:
games_df = pd.concat(games, ignore_index=True)
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463,265.0,265.0,46.0,106.0,0.434,14.0,50.0,0.280,13.0,24.0,...,30.6,29.5,181.0,112.0,GSW,114,0,2023,2023-02-01,True
5464,240.0,240.0,32.0,89.0,0.360,9.0,43.0,0.209,18.0,21.0,...,33.7,37.2,158.0,101.0,GSW,115,1,2023,2023-03-02,False
5465,240.0,240.0,39.0,93.0,0.419,14.0,46.0,0.304,23.0,27.0,...,42.9,28.1,136.0,124.0,LAC,91,0,2023,2023-03-02,True
5466,240.0,240.0,49.0,85.0,0.576,12.0,27.0,0.444,16.0,18.0,...,66.7,36.6,198.0,135.0,SAC,127,1,2023,2022-12-28,False


In [12]:
games_df.to_csv("nba_games2.csv")


In [13]:
nulls= pd.isnull(games_df)
nulls= nulls.sum()
nulls=nulls[nulls>0]
nulls

+/-            5468
mp_max         5468
mp_max         5468
+/-_max          15
+/-_opp        5468
mp_max_opp     5468
mp_max_opp     5468
+/-_max_opp      15
dtype: int64